In [1]:
def loadDataSet(fileName) :
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines() :
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

def selectJrand(i, m) :
    j = i
    while (j==i) :
        j = int(random.uniform(0,m))
        print(j)
    return j
def clipAlpha(aj,H,L) :
    if aj> H :
        aj = H
    if L > aj :
        aj = L
    return aj

In [2]:
from numpy import *
int(random.uniform(0,3))

2

In [3]:
help(random.uniform)

Help on built-in function uniform:

uniform(...) method of mtrand.RandomState instance
    uniform(low=0.0, high=1.0, size=None)
    
    Draw samples from a uniform distribution.
    
    Samples are uniformly distributed over the half-open interval
    ``[low, high)`` (includes low, but excludes high).  In other words,
    any value within the given interval is equally likely to be drawn
    by `uniform`.
    
    Parameters
    ----------
    low : float or array_like of floats, optional
        Lower boundary of the output interval.  All values generated will be
        greater than or equal to low.  The default value is 0.
    high : float or array_like of floats
        Upper boundary of the output interval.  All values generated will be
        less than high.  The default value is 1.0.
    size : int or tuple of ints, optional
        Output shape.  If the given shape is, e.g., ``(m, n, k)``, then
        ``m * n * k`` samples are drawn.  If size is ``None`` (default),
        

In [4]:
selectJrand(3,6)

3
1


1

In [3]:
import numpy as np
np.info(np.multiply)

multiply(x1, x2, /, out=None, *, where=True, casting='same_kind', order='K', dtype=None, subok=True[, signature, extobj])

Multiply arguments element-wise.

Parameters
----------
x1, x2 : array_like
    Input arrays to be multiplied.
out : ndarray, None, or tuple of ndarray and None, optional
    A location into which the result is stored. If provided, it must have
    a shape that the inputs broadcast to. If not provided or `None`,
    a freshly-allocated array is returned. A tuple (possible only as a
    keyword argument) must have length equal to the number of outputs.
where : array_like, optional
    Values of True indicate to calculate the ufunc at that position, values
    of False indicate to leave the value in the output alone.
**kwargs
    For other keyword-only arguments, see the
    :ref:`ufunc docs <ufuncs.kwargs>`.

Returns
-------
y : ndarray
    The product of `x1` and `x2`, element-wise. Returns a scalar if
    both  `x1` and `x2` are scalars.

Notes
-----
Equivalent to

In [6]:
dataArr * dataArr[1,:].T

NameError: name 'dataArr' is not defined

smo 알고리즘은 each iter 에 대해 목적함수를 최소화 시켜주는 거죠. 여기에 각 iter마다 2개의 points가 사용이 됩니다. each step에서 smo 알고리즘은 alpha i 와 alpha j를 선택하고 최적화를 진행하죠, 다른 value를 고정시켜놓고 두 개의 parameter의 최적값을 찾는 겁니다. 그렇게 alpha vector를 업데이트 해 나가는 거죠
두 개의 point는 heuristic 하게 결정됩니다 (걍 랜덤하게 결정된다고 보면 되는 것 같아요)


In [29]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    # toler : 초평면 판별함수에 그 데이터 값을 넣었을 때 실제 값과 사용자 지정
    dataMatrix = mat(dataMatIn); labelMat = mat(classLabels).transpose()
    #dataMatrix : data의 각각의 vector 값 (2개;x1, x2); labelMat : 각각의 data가 1로분류되는지 -1로 분류되는지 
    #->transpose 해서 각각의 data가 1인지 -1인지 보기 쉽게 함. row별로
    b = 0 ; m, n = shape(dataMatrix)
    # wx + b 라는 경계라인의 b값을 초기에 0으로 설정, m값은 데이터의 갯수, n은 2
    alphas = mat(zeros((m,1)))
    # 모든 데이터의 alpha값을 초기에 0으로 설정
    iter = 0
    while (iter < maxIter) :
        alphaPairsChanged = 0
        for i in range(m) :
            print("index = ",i)
            fXi = float(multiply(alphas, labelMat).T* (dataMatrix*dataMatrix[i,:].T))+b
            print("prediction=",fXi)
            #our predcition
            Ei = fXi - float(labelMat[i])
            print("error=",Ei)
            # error 
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei >toler) and (alphas[i] >0)) :
              # alpha가 0과 C 사이에 있을 때에 알파값을 업데이트 해줍니다. 알파가 0이거나 C일때에는 그 데이터가 
             # 정확히 margin line에 걸리는 값이기 때문에 굳이 alpha값을 업데이트 해줄 필요가 없는거죠
             # toler는 왜 설정하는 건지 잘 모르겠습니다ㅠㅠ
             
                j = selectJrand(i,m)
                #random한 alpha index 고르기
                fXj = float(multiply(alphas, labelMat).T * (dataMatrix*dataMatrix[j,:].T))+b
                print("j index = ",j, fXj)
                Ej = fXj - float(labelMat[j])
                print("j Error =",Ej)
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy()
                if (labelMat[i] != labelMat[j]) :
                    L= max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else :
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H : print ("L==H"); continue
                eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T
                if eta >= 0 : print ("eta>=0") ; continue
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)
                print(alphas[j])
                if (abs(alphas[j] - alphaJold) < 0.00001) : print( "j not moving enough"); continue
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])
                print(alphas[i])
                b1 = b - Ei - labelMat[i]*(alphas[i] - alphaIold)*dataMatrix[i, :]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b- Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T 
                if (0 < alphas[i]) and (C > alphas[i]) : b=b1
                elif (0 <alphas[j]) and (C>alphas[j]) : b = b2
                else : b = (b1+b2)/2.0
                alphaPairsChanged += 1
                print ("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            if (alphaPairsChanged ==0) : iter+=1
            else : iter=0
            print ("iteration number : %d" %iter)
        return b, alphas
                    

In [30]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

index =  0
prediction= 0.0
error= 1.0
76
j index =  76 0.0
j Error = 1.0
L==H
index =  1
prediction= 0.0
error= 1.0
31
j index =  31 0.0
j Error = -1.0
[[0.07206264]]
[[0.07206264]]
iter: 0 i:1, pairs changed 1
iteration number : 0
index =  2
prediction= [[1.1300134]]
error= [[0.1300134]]
iteration number : 0
index =  3
prediction= [[-1.01704276]]
error= [[-0.01704276]]
iteration number : 0
index =  4
prediction= [[0.99351297]]
error= [[-0.00648703]]
6
j index =  6 [[1.07459093]]
j Error = [[0.07459093]]
L==H
index =  5
prediction= [[0.89918734]]
error= [[-0.10081266]]
85
j index =  85 [[-1.04802437]]
j Error = [[-0.04802437]]
[[0.00197467]]
[[0.00197467]]
iter: 0 i:5, pairs changed 2
iteration number : 0
index =  6
prediction= [[1.18439934]]
error= [[0.18439934]]
iteration number : 0
index =  7
prediction= [[-0.92301533]]
error= [[0.07698467]]
13
j index =  13 [[1.1759506]]
j Error = [[0.1759506]]
[[0.]]
j not moving enough
index =  8
prediction= [[-0.61847958]]
error= [[0.38152042]]


error= [[0.96375275]]
iteration number : 0
index =  92
prediction= [[1.84204983]]
error= [[0.84204983]]
iteration number : 0
index =  93
prediction= [[-1.54925852]]
error= [[-0.54925852]]
iteration number : 0
index =  94
prediction= [[-0.80230929]]
error= [[0.19769071]]
58
j index =  58 [[-1.7198472]]
j Error = [[-0.7198472]]
L==H
index =  95
prediction= [[-1.07792967]]
error= [[-0.07792967]]
iteration number : 0
index =  96
prediction= [[-0.81290697]]
error= [[0.18709303]]
43
j index =  43 [[-1.05817496]]
j Error = [[-0.05817496]]
L==H
index =  97
prediction= [[-0.75977062]]
error= [[0.24022938]]
80
j index =  80 [[2.27472866]]
j Error = [[1.27472866]]
[[0.]]
j not moving enough
index =  98
prediction= [[-1.4795001]]
error= [[-0.4795001]]
iteration number : 0
index =  99
prediction= [[-1.2907648]]
error= [[-0.2907648]]
iteration number : 0


In [18]:
dataArr, labelArr = loadDataSet('svm.txt')

In [21]:
alphas[alphas>0]

matrix([[0.06350266, 0.0108751 , 0.01050272, 0.06610241, 0.01483752,
         0.01483752, 0.00879331, 0.00879331, 0.01050272, 0.0108751 ,
         0.00259975]])

In [22]:
for i in range(100):
    if alphas[i]>0.0 : print (i,dataArr[i], labelArr[i])

0 [3.542485, 1.977398] -1.0
3 [2.114999, -0.004466] -1.0
10 [3.125951, 0.293251] -1.0
13 [7.139979, -2.329896] 1.0
25 [7.40786, -0.121961] 1.0
30 [3.023938, -0.057392] -1.0
39 [8.804461, 3.044301] 1.0
52 [2.893743, -1.643468] -1.0
57 [6.016004, -3.753712] 1.0
62 [7.27946, -0.189572] 1.0
70 [-0.236713, -5.766721] -1.0


In [ ]:
 float(multiply(alphas, labelMat).T* (dataMatrix*dataMatrix[i,:].T))+b


In [27]:
class optStruct : 
    def __init__(self,dataMatIn, classLabels, C, toler) :
        self.X = dataMatIn
        self.labelMat = calssLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2)))
        # first column : a flag bit stating whether the eCache is valid
        # second column : the actual E value
    def calcEk(oS, k):
        fXk = float(multiply(oS.alphas, oS.labelMat).T*(oS.X*oS.X[k,:].T))+oS.b
        Ek = fXk - float(oS.labelMat[k])
        return Ek
    # 
    def selectJ(i, oS, Ei) :
        maxK = -1; maxDeltaE = 0 ; Ej = 0
        oS.eCache[i] = [1,Ei]
        validEcacheList = nonzero(oS.eCache[:,0].A)[0]
        print(validEcacheList)
        if(len(validEcacheList))>1:
            for k in validEcacheList:
                if k==i : continue
                Ek = calcEk(oS, k)
                deltaE = abs(Ei - Ek)
                if (deltaE > maxDeltaE) : 
                    maxK = k; maxDeltaE = deltaE; Ej=Ek
            return maxK, Ej
        else:
            j = selectJrand(i, oS.m)
            Ej = calcEk(oS,j)
        return j, Ej
    def updateEk(oS, k) :
        Ek = calcEk(oS, k)
        oS.eCache[k] = [1,Ek]

In [28]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei) #this has been changed from selectJrand
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: print ("L==H"; return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j] #changed for kernel
        if eta >= 0: print "eta>=0"; return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j) #added this for the Ecache
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print "j not moving enough"; return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
        updateEk(oS, i) #added this for the Ecache                    #the update is in the oppostie direction
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

def smoP(dataMatIn, classLabels, C, toler, maxIter,kTup=('lin', 0)):    #full Platt SMO
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler, kTup)
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:   #go over all
            for i in range(oS.m):        
                alphaPairsChanged += innerL(i,oS)
                print "fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
            iter += 1
        else:#go over non-bound (railed) alphas
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print "non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
            iter += 1
        if entireSet: entireSet = False #toggle entire set loop
        elif (alphaPairsChanged == 0): entireSet = True  
        print "iteration number: %d" % iter
    return oS.b,oS.alphas

def calcWs(alphas,dataArr,classLabels):
    X = mat(dataArr); labelMat = mat(classLabels).transpose()
    m,n = shape(X)
    w = zeros((n,1))
    for i in range(m):
        w += multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-28-ba42f6298f22>, line 12)